# Welcome to the Car Rental Scam Report Chats Notebook

I was recently asked to pay for damage on a car rental for damage. They car rental company produced photos of the damage and said these were incurred while the vehicle was under my possession.

Everyone tells me to take photos of my rental at pick up, I had no idea why until this moment. Upon inspecting the original photos, I found that the damage was all present on the original photos as well. **You should take photos of your rental before you drive off with it**.

The rental company saw the photos and said they wouldn't pursue me for the damage. Absolutely nuts.

In this workshop, we're going to use **Firecrawl** to scrape website contents, use **Chonkie**'s recursive chunking strategy to chunk our contents, store the chunks as embeddings in **Weaviate**, and then chat with our data through a simple RAG chatbot powered by **OpenAI**.

## Install the Firecrawl dependency

In [1]:
!pip install firecrawl-py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from firecrawl import FirecrawlApp, ScrapeOptions

import getpass
FIRECRAWL_API_KEY = getpass.getpass("Enter your firecrawl API key: ")

app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

Enter your firecrawl API key:  ········


In [23]:
source_url = 'https://forums.moneysavingexpert.com/discussion/6437902/zipcar-false-fine'
# Scrape a website:
scrape_status = app.scrape_url(
  source_url, 
  formats=['markdown', 'html']
)
print(scrape_status.markdown)


MSE Links

- ![](https://us-noi.v-cdn.net/6031891/uploads/editor/uo/6gfujhszuwjp.png)

[Parking ticket appeals\\
\\
MSE's guide helps you reclaim all unfair council, police or TFL parking fines](https://www.moneysavingexpert.com/reclaim/parking-ticket-appeals/?from=forum-guidesbar-parking-tickets-fines-parking)

- ![](https://us-noi.v-cdn.net/6031891/uploads/editor/kd/7basbdtoqak4.png)

[How to get cheap airport parking\\
\\
Slash the cost of airport parking, even on the day, using this MoneySavingExpert guide](https://www.moneysavingexpert.com/reclaim/parking-ticket-appeals/?from=forum-guidesbar-parking-tickets-fines-parking)


We'd like to remind Forumites to please **avoid political debate** on the Forum... Read More »
This is to keep it a safe and useful space for MoneySaving discussions. Threads that are - or become - political in nature may be removed in line with the [Forum’s rules](https://www.moneysavingexpert.com/site/forum-faqs/). Thank you for your understanding.


**IMPORT

## Install Chonkie dependency

In [6]:
!pip install chonkie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chonkie: filename=chonkie-1.1.1-cp312-cp312-macosx_10_9_universal2.whl size=409379 sha256=4d9fe30acee45d45fd7cd84ce7cb9a0967b566ed9ed138f6c0e98facd987721a
  Stored in directory: /Users/ajchan/Library/Caches/pip/wheels/83/cb/7e/f95a043078c06d2084804dcbdf4954516b03194f5857d99fca
Successfully built chonkie

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Use RecursiveChunker to chunk our scraped website

In [44]:
# First import the chunker you want from Chonkie
from chonkie import RecursiveChunker

# Initialize the chunker
chunker = RecursiveChunker(
    chunk_size = 512
)

chunks = chunker(scrape_status.markdown)

# Access chunks
for chunk in chunks:
    print(f"\n\n\n\n\Chunk: {chunk.text}")
    print(f"Tokens: {chunk.token_count}")






\Chunk: MSE Links

- ![](https://us-noi.v-cdn.net/6031891/uploads/editor/uo/6gfujhszuwjp.png)

[Parking ticket appeals\\
\\
MSE's guide helps you reclaim all unfair council, police or TFL parking fines](https://www.moneysavingexpert.com/reclaim/parking-ticket-appeals/?from=forum-guidesbar-parking-tickets-fines-parking)

- ![](https://us-noi.v-cdn.net/6031891/uploads/editor/kd/7basbdtoqak4.png)

[How to get cheap airport parking\\

Tokens: 426




\Chunk: \\
Slash the cost of airport parking, even on the day, using this MoneySavingExpert guide](https://www.moneysavingexpert.com/reclaim/parking-ticket-appeals/?from=forum-guidesbar-parking-tickets-fines-parking)


We'd like to remind Forumites to please **avoid political debate** on the Forum... Read More »

Tokens: 306




\Chunk: This is to keep it a safe and useful space for MoneySaving discussions. Threads that are - or become - political in nature may be removed in line with the [Forum’s rules](https://www.moneysavingexpert.com/s

<>:13: SyntaxWarning: invalid escape sequence '\C'
<>:13: SyntaxWarning: invalid escape sequence '\C'
/var/folders/t4/lrgxkdbn5cx0v5m9slyg9d300000gn/T/ipykernel_91076/3643748144.py:13: SyntaxWarning: invalid escape sequence '\C'
  print(f"\n\n\n\n\Chunk: {chunk.text}")


## Install Weaviate Dependency

In [10]:
!pip install weaviate-client

  Using cached validators-0.35.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached authlib-1.6.1-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached cryptography-45.0.5-cp311-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached protobuf-6.31.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.2/597.2 kB 5.1 MB/s eta 0:00:00a 0:00:01
Using cached authlib-1.6.1-py2.py3-none-any.whl (239 kB)
Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 36.6 MB/s eta 0:00:0000:010:01
Using cached validators-0.35.0-py3-none-any.whl (44 kB)
Using cached protobuf-6.31.1-cp39-abi3-macosx_10_9_universal2.whl (425 kB)
Using cached cryptography-45.0.5-cp311-abi3-macosx_10_9_universal2.whl (7.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf

## Set up a Weaviate Client

In [11]:
WEAVIATE_CLUSTER_URL = "fe1rykarx6s5nhtw6hq5g.c0.us-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY = getpass.getpass("Enter your Weaviate API key: ")
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")


Enter your Weaviate API key:  ········
Enter your OpenAI API key:  ········


In [17]:
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes as wvc
import weaviate.classes.config as wc


weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_CLUSTER_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
    }
)

weaviate_client.is_ready()

True

## Create a Collection in Weaviate

In [45]:
if input("Do you want to delete and recreate the CarRentalChunks collection? (y/n): ") == 'y':
    weaviate_client.collections.delete("CarRentalChunks")
    print("Deleted CarRentalChunks collection")

    car_rental_collection = weaviate_client.collections.create(
        name="CarRentalChunks",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_weaviate(model="Snowflake/snowflake-arctic-embed-l-v2.0"),
        generative_config=wvc.config.Configure.Generative.openai(model="gpt-4o-mini"),
        properties=[
            wc.Property(name="source_url", data_type=wc.DataType.TEXT),
            wc.Property(name="content", data_type=wc.DataType.TEXT),
        ],
    )

    print("Created CarRentalChunks collection")


Do you want to delete and recreate the CarRentalChunks collection? (y/n):  y


Deleted CarRentalChunks collection
Created CarRentalChunks collection


In [48]:
with car_rental_collection.batch.fixed_size(batch_size=200) as batch:
    for chunk in chunks:
        batch.add_object(
            {
                "content": chunk.text,
                "source_url": source_url
            }
        )
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break




In [52]:

def chat_with_car_rental_data():
    """Interactive chat interface for querying car rental data."""
    car_rental_chunk_collection = weaviate_client.collections.get("CarRentalChunks")
    
    print("🚗 Car Rental Data Chat Interface")
    print("Ask questions about car rental experiences, issues, or advice!")
    print("Type 'quit' or 'exit' to end the conversation.\n")
    
    while True:
        try:
            # Get user input
            user_query = input("\n\n\n\n\n❓ Your question: ").strip()
            
            if user_query.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Thanks for chatting! Goodbye!")
                break
            
            if not user_query:
                print("Please enter a question.")
                continue
            
            print("\n🔍 Searching relevant information...")
            
            # Search for relevant chunks
            search_response = car_rental_chunk_collection.query.near_text(
                query=user_query,
                limit=3,
                return_metadata=['score']
            )

            # print("Search response:")
            # print(search_response.objects)
            # print("----")

            if not search_response.objects:
                print("❌ No relevant information found. Try rephrasing your question.")
                continue
            
            # Generate response based on found chunks
            response = car_rental_chunk_collection.generate.near_text(
                query=user_query,
                limit=3,
                grouped_task=f"""Share similar stories back to the user based on the experience as defined by: 
                
                --------- BEGIN USER QUERY --------- 
                {user_query}
                -------- END USER QUERY -------
                Generate a response and include links if there are any relevant links."""
            )
            
            print("\n🤖 Answer:")
            print(response.generated)
            
            # Show source information
            print("\n📚 Sources used:")
            for i, obj in enumerate(search_response.objects[:3], 1):
                content_preview = obj.properties['content'][:100] + "..." if len(obj.properties['content']) > 100 else obj.properties['content']
                score = obj.metadata.score if hasattr(obj.metadata, 'score') else 'N/A'
                print(f"  {i}. Score: {score:.3f} - {content_preview}")
            
        except KeyboardInterrupt:
            print("\n\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again with a different question.")
    
    client.close()  # Free up resources

In [ ]:
chat_with_car_rental_data()


🚗 Car Rental Data Chat Interface
Ask questions about car rental experiences, issues, or advice!
Type 'quit' or 'exit' to end the conversation.




❓ Your question:  test



🔍 Searching relevant information...

🤖 Answer:
It seems like you're looking for stories related to experiences with Zipcar and issues surrounding fines. Here are some relevant discussions that might resonate with your query:

1. **Zipcar False Fine Discussion**: This thread discusses a user's experience with receiving a false fine from Zipcar. The community shares insights and advice on how to handle such situations. You can read more about it [here](https://forums.moneysavingexpert.com/discussion/6437902/zipcar-false-fine).

2. **User Experiences with Parking Tickets**: In the same forum, there are various discussions about parking tickets and fines, where users share their experiences and tips on contesting fines. This could provide additional context and support if you're facing a similar issue. Check out the category [here](https://forums.moneysavingexpert.com/categories/parking-tickets-fines-parking).

If you have a specific aspect of your experience you'd like to explore further


❓ Your question:  any information about car damage? 



🔍 Searching relevant information...

🤖 Answer:
It seems you're looking for information regarding car damage. Here are some relevant discussions and insights that might help you:

1. **Understanding Liability in Car Rentals**: A user on a forum mentioned the importance of reading the rental contract carefully to determine liability for any damages incurred. This is crucial when renting a vehicle, as different companies have varying policies regarding damage and fines. You can read more about this discussion [here](https://forums.moneysavingexpert.com/discussion/6437902/zipcar-false-fine).

2. **Handling False Fines**: Another thread discusses experiences with false fines related to car rentals, which can sometimes arise from misunderstandings about damage or usage. It's a good reminder to keep all documentation and communication clear. You can check out the details [here](https://forums.moneysavingexpert.com/discussion/6437902/zipcar-false-fine).

3. **General Advice on Car Damage**: I


❓ Your question:  a car rental company asking me for money. How can I protect myself?



🔍 Searching relevant information...

🤖 Answer:
If you're facing a situation where a car rental company is asking you for money, there are several steps you can take to protect yourself. Here are some similar experiences and advice shared by others:

1. **Dispute the Charge**: If you believe the charge is unjustified, consider contacting your credit card company to dispute the payment. You can ask them to put a hold on the payment while the dispute is being resolved. This can prevent the rental company from taking money from your account without your consent. 

   - Example: A user mentioned that they used a card where they only transfer money to avoid unauthorized charges. They were able to prevent the rental company from taking money by not transferring funds to that card. [Read more here](https://forums.moneysavingexpert.com/discussion/6437902/zipcar-false-fine).

2. **Check Your Insurance**: If you have home insurance, check if it includes legal expenses cover. If it does, you can 


❓ Your question:  Maintenance claims



🔍 Searching relevant information...

🤖 Answer:
It seems you're interested in stories related to maintenance claims. Here are some relevant experiences and resources that might help you navigate similar situations:

1. **Parking Ticket Appeals**: Many users have shared their experiences with unfair parking fines, including how they successfully appealed against them. If you find yourself in a situation where you believe a maintenance claim or fine is unjust, you might want to check out this guide on how to reclaim unfair council, police, or TFL parking fines. 
   - [MSE's Guide to Parking Ticket Appeals](https://www.moneysavingexpert.com/reclaim/parking-ticket-appeals/?from=forum-guidesbar-parking-tickets-fines-parking)

2. **Legal Expenses Cover**: If you have home insurance, it's worth checking if you have legal expenses cover. Many users have found that contacting the legal helpline provided by their insurer has helped them resolve disputes related to maintenance claims or other leg

In [ ]:
weaviate_client.close()